In [3]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset, IterableDataset, SubsetRandomSampler,Sampler, Subset
from dabs.src.datasets.catalog import DATASET_DICT
data_root = '/disk2/ofirb/dabs/data'
image_size = 32
train_dataset = DATASET_DICT['lfw32'](base_root=data_root, train=True)
val_dataset = DATASET_DICT['lfw32'](base_root=data_root, train=False)

train_loader = DataLoader(train_dataset,batch_size=64,num_workers=0,shuffle=False,drop_last=True)
val_loader = DataLoader(val_dataset,batch_size=64,num_workers=0,shuffle=False,drop_last=True)



In [6]:
####### COMPUTE MEAN / STD

# placeholders
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for _, images , _ in tqdm(train_loader):
    psum    += images.sum(axis        = [0, 2, 3])
    psum_sq += (images ** 2).sum(axis = [0, 2, 3])

# loop through images
for _, images , _ in tqdm(val_loader):
    psum    += images.sum(axis        = [0, 2, 3])
    psum_sq += (images ** 2).sum(axis = [0, 2, 3])

100%|██████████| 20/20 [00:01<00:00, 11.42it/s]


In [7]:
####### FINAL CALCULATIONS

# pixel count
count = (len(train_dataset)+len(val_dataset)) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4321, 0.3748, 0.3333])
std:  tensor([0.2819, 0.2563, 0.2500])
